In [1]:
from datetime import datetime
import json

In [2]:
class Task:
    def __init__(self, id, title, priority, due_date, status="Pending"):
        self.id = id
        self.title = title
        self.priority = priority
        self.due_date = due_date
        self.status = status
    
    def to_dict(self):
        return {
            "id": self.id,
            "title": self.title,
            "priority": self.priority,
            "due_date": self.due_date,
            "status": self.status
        }
    
    @staticmethod
    def from_dict(data):
        return Task(data["id"], data["title"], data["priority"], data["due_date"], data["status"])

In [3]:
class TaskManager:
    
    def __init__(self):
        self.task_list = []
        self.next_id = 1
        self.load_from_file()
    
    def add_task(self, title, priority, due_date):
        task = Task(self.next_id, title, priority, due_date)
        self.task_list.append(task)
        self.next_id += 1
        self.save_to_file()
        return task
    
    def view_tasks(self):
        return self.task_list
    
    def update_task(self, task_id, title=None, priority=None, due_date=None):
        task = self._find_task(task_id)
        if task:
            if title: task.title = title
            if priority: task.priority = priority
            if due_date: task.due_date = due_date
            self.save_to_file()
            return True
        return False
    
    def mark_complete(self, task_id):
        task = self._find_task(task_id)
        if task:
            task.status = "Completed"
            self.save_to_file()
            return True
        return False
    
    def delete_task(self, task_id):
        task = self._find_task(task_id)
        if task:
            self.task_list.remove(task)
            self.save_to_file()
            return True
        return False
    
    def filter_tasks(self, by="status", value=None):
        if by == "status":
            return [t for t in self.task_list if t.status == value]
        elif by == "due_date":
            return [t for t in self.task_list if t.due_date == value]
        return self.task_list
    
    def save_to_file(self):
        try:
            with open("tasks.json", "w") as f:
                json.dump([t.to_dict() for t in self.task_list], f, indent=2)
        except Exception as e:
            print(f"Error saving: {e}")
    
    def load_from_file(self):
        try:
            with open("tasks.json", "r") as f:
                data = json.load(f)
                self.task_list = [Task.from_dict(t) for t in data]
                self.next_id = max([t.id for t in self.task_list], default=0) + 1
        except FileNotFoundError:
            pass
        except Exception as e:
            print(f"Error loading: {e}")
    
    def _find_task(self, task_id):
        return next((t for t in self.task_list if t.id == task_id), None)

In [4]:
# Display function for tasks
def print_tasks(tasks):
    if not tasks:
        print("\nNo tasks found.")
        return
    
    print("\n" + "="*80)
    print(f"{'ID':<5} {'Title':<25} {'Priority':<10} {'Due Date':<12} {'Status':<10}")
    print("="*80)
    for task in tasks:
        print(f"{task.id:<5} {task.title:<25} {task.priority:<10} {task.due_date:<12} {task.status:<10}")
    print("-"*80)

In [6]:
def get_input(prompt, valid_options=None):
    while True:
        value = input(prompt).strip()
        if not value:
            print("Input cannot be empty.")
            continue
        if valid_options and value not in valid_options:
            print(f"Invalid input. Choose from: {', '.join(valid_options)}")
            continue
        return value

def main_console():
    manager = TaskManager()
    
    while True:
        print("\n" + "="*40)
        print("TaskForge - Task Manager")
        print("="*40)
        print("1. Add Task")
        print("2. View All Tasks")
        print("3. Filter Tasks")
        print("4. Update Task")
        print("5. Mark Task Complete")
        print("6. Delete Task")
        print("7. Exit")
        
        choice = input("\nEnter choice: ").strip()
        
        match choice:
            case "1":
                title = get_input("Title: ")
                priority = get_input("Priority (Low/Medium/High): ", ["Low", "Medium", "High"])
                due_date = get_input("Due Date (YYYY-MM-DD): ")
                try:
                    datetime.strptime(due_date, "%Y-%m-%d")
                    task = manager.add_task(title, priority, due_date)
                    print(f"\n✓ Task added with ID: {task.id}")
                except ValueError:
                    print("Invalid date format.")
            
            case "2":
                print_tasks(manager.view_tasks())
            
            case "3":
                print("\n1. Filter by Status")
                print("2. Filter by Due Date")
                filter_choice = input("Choose filter: ").strip()
                
                if filter_choice == "1":
                    status = get_input("Status (Pending/Completed): ", ["Pending", "Completed"])
                    print_tasks(manager.filter_tasks(by="status", value=status))
                elif filter_choice == "2":
                    due_date = get_input("Due Date (YYYY-MM-DD): ")
                    print_tasks(manager.filter_tasks(by="due_date", value=due_date))
            
            case "4":
                try:
                    task_id = int(input("Task ID: "))
                    if manager._find_task(task_id):
                        print("\nLeave inputs blank to keep the existing value.")
                        title = input("New Title: ").strip() or None
                        priority = input("New Priority (Low/Medium/High): ").strip() or None
                        due_date = input("New Due Date (YYYY-MM-DD): ").strip() or None
                        manager.update_task(task_id, title, priority, due_date)
                        print("\n✓ Task updated")
                    else:
                        print("\n✗ Task not found")
                except ValueError:
                    print("Invalid ID")
            
            case "5":
                try:
                    task_id = int(input("Task ID: "))
                    if manager.mark_complete(task_id):
                        print("\n✓ Task marked complete")
                    else:
                        print("\n✗ Task not found")
                except ValueError:
                    print("Invalid ID")
            
            case "6":
                try:
                    task_id = int(input("Task ID: "))
                    if manager.delete_task(task_id):
                        print("\n✓ Task deleted")
                    else:
                        print("\n✗ Task not found")
                except ValueError:
                    print("Invalid ID")
            
            case "7":
                print("\nGoodbye!")
                break
            
            case _:
                print("\nInvalid choice")

# Uncomment the line below to run the interactive console
main_console()
# if __name__ == "__main__":
#     main_console()


TaskForge - Task Manager
1. Add Task
2. View All Tasks
3. Filter Tasks
4. Update Task
5. Mark Task Complete
6. Delete Task
7. Exit

✓ Task added with ID: 2

TaskForge - Task Manager
1. Add Task
2. View All Tasks
3. Filter Tasks
4. Update Task
5. Mark Task Complete
6. Delete Task
7. Exit

ID    Title                     Priority   Due Date     Status    
1     7                         Low        2025-12-27   Pending   
2     have water                Medium     2025-12-27   Pending   
--------------------------------------------------------------------------------

TaskForge - Task Manager
1. Add Task
2. View All Tasks
3. Filter Tasks
4. Update Task
5. Mark Task Complete
6. Delete Task
7. Exit

Goodbye!


In [7]:
import os
print("Current directory:", os.getcwd())
print("Tasks file will be saved at:", os.path.join(os.getcwd(), "tasks.json"))


Current directory: /content
Tasks file will be saved at: /content/tasks.json
